In [1]:
import csv
import numpy as np
from datetime import datetime as dt

In [2]:
data_paths = [
    '2023-07-05_raw.csv',
    '2023-07-06_raw.csv',
    '2023-07-07_raw.csv',
    '2023-07-08_raw.csv',
    '2023-07-09_raw.csv',
    '2023-07-10_raw.csv',
    '2023-07-11_raw.csv',
    '2023-07-12_raw.csv',
    '2023-07-13_raw.csv',
    '2023-07-14_raw.csv',
    '2023-07-15_raw.csv'
]

date_data = []
temp_data = []
humd_data = []
power_data = []

for path in data_paths:
    date_data_t = []
    temp_data_t = []
    humd_data_t = []
    power_data_t = []
    with open(f'zuraach_ail2/{path}', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            d, t, h, p = row
            d = dt.fromisoformat(d)
            d = d.hour + d.minute / 60
            date_data_t.append(d)
            temp_data_t.append(float(t))
            humd_data_t.append(float(h))
            power_data_t.append(float(p))
    date_data.extend(date_data_t)
    temp_data.extend(temp_data_t)
    humd_data.extend(humd_data_t)
    power_data.extend(power_data_t)
dn = np.array(date_data) / 24
tn = np.array(temp_data)
print(tn.max())
tn /= tn.max()
hn = np.array(humd_data)
print(hn.max())
hn /= hn.max()
pn = np.array(power_data) / 1.8

dn_list = dn.tolist()
tn_list = tn.tolist()
hn_list = hn.tolist()
pn_list = pn.tolist()

34.46
54.88


In [3]:
WINDOW = 7

data_batch = []
label = []
for i in range(len(dn_list) - (WINDOW - 1)):
    data = []                               #  shape(4, 6) ((feature, label), seq)
    drop = False
    label = 0
    if pn_list[i:i+WINDOW][0] >= 0.6:
        label = 1
    else:
        if np.random.random() <= 1:  # 0.14
            label = 0
        else:
            drop = True
    if not drop:
        data.append(dn_list[i:i+WINDOW])
        data.append(tn_list[i:i+WINDOW])
        data.append(hn_list[i:i+WINDOW])
        data.append([label for _ in range(WINDOW)])
        data_T = np.array(data).T            #  shape(6, 4) (seq, (feature, label))
        data_batch.append(data_T)
data_batch = np.array(data_batch)            #  shape(N, seq, (feature, label))

In [4]:
label, count = np.unique(data_batch[:, :, -1][:, 0], return_counts=True)
label, count / count.sum(), count

(array([0., 1.]),
 array([0.8806728, 0.1193272]),
 array([2775,  376], dtype=int64))

In [5]:
np.save('dataset_all.npy', data_batch)
len(data_batch)

3151